<a href="https://colab.research.google.com/github/nikeyes/GenAI-experiments/blob/main/query-my-documents/ChatGPT-with-my-own-information.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install llama-index

In [ ]:
import os
from configparser import ConfigParser

parser = ConfigParser()
_ = parser.read('config.ini')
os.environ["OPENAI_API_KEY"] = parser.get('OpenAI','API_KEY')

# Parseamos el PDF a modo de ejemplo. 


## Si fuera un caso real usáriamos librerias más complejas que usen OCR para leer texto en imágenes, que sepan segmentar el documento .... 
Algo como LangChain con unstructured_file : https://python.langchain.com/en/latest/modules/indexes/document_loaders/examples/unstructured_file.html

In [ ]:
from pathlib import Path
from llama_index import download_loader

PDFReader = download_loader("PDFReader")

loader = PDFReader()
#documents = loader.load_data(file=Path('./Adevinta-Q4-2022-Quarterly-report.pdf'))
documents = loader.load_data(file=Path('./Adevinta-Annual-Report-2022.pdf'))

In [ ]:
#documents[0].get_text().find('Adevinta ASA’s Board of Directors')
documents[0].get_text().find('Tonnes of carbon') -1
#documents[0].get_text()

80323

# Generamos el índice de Embedings y lo guardamos para no volver a pagar la API de Embedings

In [ ]:
from llama_index import GPTVectorStoreIndex

index = GPTVectorStoreIndex.from_documents(documents)

index.storage_context.persist(persist_dir="data_annual")

# Cargamos el índice desde local

In [ ]:
from llama_index import StorageContext, load_index_from_storage

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="data_annual")

# load index
index = load_index_from_storage(storage_context)

# Consultamos a ChatGPT sobre nuestro documento Adevinta-Annual-Report-2022.pdf

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("What is the total revenue of Adevinta in 2022?")
print(response)


€1,644 million


In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("What is the Number of shares non-vested at 31 December 2022?")
print(response)


3,915,694


In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("List the Adevinta’s shareholders and their percent of shares outstanding")
print(response)


Rank Name % of shares outstanding 
1 eBay Inc.* 33.0% 
2 Schibsted ASA 28.1% 
3 Astinlux Finco S.à r.l. 11.2% 
4 Danske Bank A/S 3.0% 
5 Baillie Gifford & Co. 2.8% 
6 Folketrygdfondet 2.1% 
7 Blommenholm Industrier AS 1.4% 
8 Capital World Investors 1.0% 
9 Vor Capital LLP. 0.9% 
10 The Vanguard Group, Inc. 0.8% 
11 DNB Asset Management AS 0.7% 
12 Premier Miton Investors 0.7% 
13 BlackRock Institutional Trust Company, N.A. 0.7% 
14 KLP Fondsforvaltning AS 0.5% 
15 Eminence Capital, LP 0.5% 
16 Fidelity Management & Research Company LLC 0.5% 
17 Alecta pensionsförsäkring, ömsesidigt 0.5% 
18 Storebrand Kapitalforval


## Ejemplo de error...

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("List the names of Adevinta ASA’s Board of Directors and the number of their shares held in Adevinta")
print(response)


Adevinta ASA does not list the names of its Board of Directors or the number of shares held in Adevinta. This information is not provided in the context information.


In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("How many Tonnes of carbon emissions were potentially saved arising from the transactions on our platforms")
print(response)



The exact amount of carbon emissions saved arising from the transactions on our platforms cannot be determined from the given context information. However, according to the information, users from our eight participating Adevinta sites contributed to potential savings in greenhouse gas emissions and the use of materials associated with choosing second-hand goods over new goods. Additionally, the Myanmar Stoves Campaign distributed 49,000 stoves to around 225,000 beneficiaries, which reduced wood consumption by 50%, indoor air pollution by 80%, and carbon emissions by 60% per stove.


In [ ]:
#import logging
#import sys

#logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
#logger = logging.getLogger('LOGGER_NAME')

query_engine = index.as_query_engine()
response = query_engine.query("How many tonnes of carbon emissions were potentially saved as a result of the transactions conducted on our platforms?")
print(response)


It is not possible to answer this question without further information. The context information provided does not include any data on the number of transactions conducted on our platforms or the amount of carbon emissions potentially saved as a result of these transactions.



# Consultamos a ChatGPT sobre nuestro documento Adevinta-Q4-2022-Quarterly-report.pdf


In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("What is the adjusted net cash flow from operating activities?")
print(response)


The adjusted net cash flow from operating activities is 114 million Euros for the fourth quarter of 2022 and 437 million Euros for the year 2022.


In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("write an analisys of the adjusted net cash flow from operating activities?")
print(response)


The adjusted net cash flow from operating activities is a useful indicator of the amount of cash flows generated by operating activities, after income tax. It is calculated by taking EBITDA, plus the decrease or minus the increase in non-cash items (including share-based compensation), change in working capital and provisions related to EBITDA, minus the payment of income tax, minus development and purchase of property, plant and equipment and intangible assets, and minus IFRS 16 lease payments. This measure enables an evaluation of operating performance and provides an indicator of the overall strength of the company. 

In the fourth quarter of 2022, the adjusted net cash flow from operating activities was €114 million, compared to €72 million in the fourth quarter of 2021. This increase of €42 million was mainly driven by an increase in EBITDA of €21 million, a decrease in non-cash items of €2 million, and a decrease in the payment of income tax of €21 million. For the full year 202

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("Taken into account all the information of Adevinta-Q4-2022-Quarterly-report.pdf, is Adevinta a good company to invest in?")
print(response)


It is difficult to answer this question without prior knowledge of Adevinta's financial performance and other factors that may influence the decision to invest in the company. However, based on the information provided in the quarterly report, Adevinta appears to have a strong focus on cost optimization and financial discipline, as well as a product roadmap that is aligned with its strategic ambitions. Additionally, the company has reported a 2-point increase in EBITDA margin in the fourth quarter, and has generated strong cash flow, allowing it to further repay debt and mitigate some of the impact of rising interest rates. These factors may indicate that Adevinta is a good company to invest in.
